In [27]:
import pandas as pd
from tqdm import tqdm
from loguru import logger
import sys
import os
import gc
import matplotlib.pyplot as plt
import seaborn as sns

%load_ext autoreload
%autoreload 2
sys.path.append("/Users/xinc./Documents/GitHub/note")
sys.path.append(os.getcwd()) # 加入目前路徑以匯入 utils

from module.get_info_Postgre import PostgresClient
from module.get_info_FinMind import FinMindClient
from module.plot_func import plot
from utils import batch_fetch_prices, run_event_study

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# read

In [2]:
# 讀取資料庫中的 disposal_info 表格
database_name = "disposal"
client = PostgresClient(database = database_name)
disposal_info = client.fetch_table(table ='disposal_info', date_col='date', start_date='2018-01-01', end_date='2025-12-31', order_by='date')
disposal_info = disposal_info.loc[disposal_info["is_disposal"] == True]

In [3]:
disposal_info

,date,stock_id,is_disposal
121,2018-01-01,1475,True
289,2018-01-01,2424,True
1235,2018-01-01,3219,True
1493,2018-01-01,911616,True
1506,2018-01-01,8287,True
...,...,...,...
7641466,2025-11-11,2515,True
7641784,2025-11-11,8358,True
7641952,2025-11-11,6488,True
7642278,2025-11-11,3163,True


In [5]:
# [Optimization] 使用平行化抓取加速下載
# 關閉 Log 避免洗版
logger.remove()
logger.add(sys.stderr, level="WARNING")

fm_client = FinMindClient()
price_df = batch_fetch_prices(fm_client, disposal_info, max_workers=10)

if not price_df.empty:
    print(price_df.head())
else:
    print("No price data fetched.")

gc.collect()


Found 895 unique stocks with disposal events.


Fetching by Stock: 100%|██████████| 895/895 [02:50<00:00,  5.25it/s]

Fetched 33450 price rows.
        Date Stock_id   Open   High    Low  Close  Volume  TradingAmount
0 2017-12-29     1475  48.00  53.30  48.00  50.00  901560       45481144
1 2018-01-02     1475  48.20  48.20  45.00  45.00  169974        7779259
2 2018-01-03     1475  45.10  48.00  45.10  45.85   87200        4039709
3 2018-01-04     1475  46.95  47.95  46.00  46.80   65042        3053813
4 2018-01-05     1475  46.00  46.30  45.75  46.05   52010        2386158


0

## Analysis: Event Study on Disposal Effect

In [30]:
# --- Event Study Analysis (Using Optimized Module) ---

event_study_df = run_event_study(price_df, disposal_info)

if not event_study_df.empty:
    print(event_study_df[['Date', 'Stock_id', 't_label', 'gap_days', 'calendar_relative_day', 'Close']].head(10))
    
    # 寫入 CSV 方便檢查
    event_study_df.to_csv('test.csv', index=False)
else:
    print("Analysis returned empty DataFrame.")

Running Event Study Analysis...
Original events: 36512, Consolidated Start Events: 2192


KeyError: "['Close', 'Volume'] not in index"